In [67]:
from pathlib import Path
import pandas as pd
import altair as alt
import seaborn as sns

In [68]:
Path.cwd()

PosixPath('/Users/ben/Developer/SpaceMissionDES/SpaceMissionDES/analysis')

#### Load the Data

In [69]:
data_dir = Path.cwd().parent.parent / "results" / "test-01"

mc_results = pd.read_csv(data_dir / "mc.csv")
mc_results.head()

,replicant,outcome,duration,anomaly_count,anomaly_time,anomaly_vehicle,anomaly_activity
0,0,True,190.0,0,NaN,NaN,NaN
1,1,True,190.0,0,NaN,NaN,NaN
2,2,False,190.0,2,40.0,Tanker,Ascent
3,2,False,190.0,3,190.0,Tanker,PropTransfer
4,4,False,550.0,2,40.0,Tanker,Ascent


#### Summary Statistics

In [70]:
N = mc_results.replicant.max()
success_rate = sum(mc_results.groupby("replicant").first().outcome) / N
print(f"The success rate was: {success_rate} = {100*success_rate:.3f} %\n")

The success rate was: 0.5775775775775776 = 57.758 %



#### Mission Duration

In [71]:
df = mc_results.groupby("replicant", as_index=False).first()[["replicant", "duration"]]

alt.Chart(df).mark_bar().encode(
    # alt.X("duration:Q", bin=True),
    alt.X("duration:Q", bin=alt.Bin(extent=[0, 4000], step=250)),
    y='count()',
).properties(
    width = 800
)

alt.Chart(...)

Compute the Emprical CDF

In [78]:
import numpy as np

# from: https://stackoverflow.com/questions/33345780/empirical-cdf-in-python-similiar-to-matlabs-one
def ecdf(sample):

    # convert sample to a numpy array, if it isn't already
    sample = np.atleast_1d(sample)

    # find the unique values and their corresponding counts
    quantiles, counts = np.unique(sample, return_counts=True)

    # take the cumulative sum of the counts and divide by the sample size to
    # get the cumulative probabilities between 0 and 1
    cumprob = np.cumsum(counts).astype(np.double) / sample.size

    return quantiles, cumprob

def plot_ECDF(df):
    sample = df.duration.values
    qe, pe = ecdf(sample)
    data = pd.DataFrame({"duration": list(qe), "prob": list(pe)})
    fig = alt.Chart(data).mark_line().encode(x = "duration", y = "prob").properties(width=500, height=250)
    return fig


In [79]:
plot_ECDF(df)

alt.Chart(...)

#### Failure Diagnostics (Aggregate)

Which Vehicles Fail

In [120]:
# Relative failure frequency by vehicle
df = (
    mc_results
    .groupby("anomaly_vehicle")
    .count()
    .rename(columns={'replicant': 'n'})
    .loc[:,'n']
)

fail_count = df.sum()
# (df / fail_count).reset_index()

alt.Chart(
    (df / fail_count).reset_index()
).mark_bar().encode(
    y = "anomaly_vehicle",
    x = "n"
)

alt.Chart(...)

Which Activities Fail?

In [121]:
# Relative failure frequency by activity
df = (
    mc_results
    .groupby("anomaly_activity")
    .count()
    .rename(columns={'replicant': 'n'})
    .loc[:,'n']
)

fail_count = df.sum()
# (df / fail_count).reset_index()

alt.Chart(
    (df / fail_count).reset_index()
).mark_bar().encode(
    y = "anomaly_activity",
    x = "n"
)

alt.Chart(...)

#### Failure Diagnostics (per mission)

Avg ascent failures per mission

In [122]:
mc_results.head()

,replicant,outcome,duration,anomaly_count,anomaly_time,anomaly_vehicle,anomaly_activity
0,0,True,190.0,0,NaN,NaN,NaN
1,1,True,190.0,0,NaN,NaN,NaN
2,2,False,190.0,2,40.0,Tanker,Ascent
3,2,False,190.0,3,190.0,Tanker,PropTransfer
4,4,False,550.0,2,40.0,Tanker,Ascent


In [181]:
# mc_results.query("anomaly_count > 4")
df = mc_results.query("replicant == 3")
df

,replicant,outcome,duration,anomaly_count,anomaly_time,anomaly_vehicle,anomaly_activity
15,3,False,1030.0,2,40.0,Tanker,Ascent
16,3,False,1030.0,3,160.0,Tanker,Ascent
17,3,False,1030.0,4,280.0,Tanker,Ascent
18,3,False,1030.0,5,400.0,Tanker,Ascent
19,3,False,1030.0,6,520.0,Tanker,Ascent
20,3,False,1030.0,7,640.0,Tanker,Ascent
21,3,False,1030.0,8,760.0,Tanker,Ascent
22,3,False,1030.0,9,1030.0,MoonShip,TranslunarCoast


In [178]:
df.anomaly_activity.value_counts()

Ascent             789
PropTransfer       188
Rendezvous         101
Disposal            68
TranslunarCoast     66
Name: anomaly_activity, dtype: int64

In [179]:
df = mc_results.groupby("replicant", as_index=False)["anomaly_activity"].value_counts()
df

,replicant,anomaly_activity,count
0,2,PropTransfer,1
1,2,Ascent,1
2,3,Ascent,7
3,3,TranslunarCoast,1
4,4,Ascent,4
...,...,...,...
1207,996,Ascent,10
1208,996,Disposal,1
1209,997,Ascent,5
1210,998,Ascent,20


In [184]:
# alt.Chart(df.unstack().Ascent.reset_index()).mark_boxplot().encode(x = "Ascent")

In [185]:
df = mc_results.groupby("replicant", as_index=False)["anomaly_activity"].value_counts()

alt.Chart(df).mark_boxplot().encode(
    x = "count",
    y = "anomaly_activity"

)

alt.Chart(...)